# Create a FABRIC Facility Port

This notebook shows how to use create an facility port to connect your FABRIC experiment to an external facility. 



## Import the FABlib Library

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
fablib.show_config();

## Create the Experiment Slice

The following creates a single node with basic compute capabilities. You build a slice by creating a new slice and adding resources to the slice. After you build the slice, you must submit a request for the slice to be instantiated.   

By default, the submit function will block until the node is ready and will display the progress of your slice being built.



In [ ]:
slice_name = "Slice-AWSDX-SENSE"

facility_port_site='MASS'
facility_port_name = 'SENSE-MGHPCC' # VLAN Range: 3914-3920


### Facility port info

#### Find the facility ports on the site
List the facility ports
- Facility Port Name
- Available VLAN range
- Currently allocated VLAN range

In [ ]:
cell_output = 'pandas' # 'json'
output_list = fablib.list_facility_ports(filter_function=lambda x: x['name'] == facility_port_name, output=cell_output)

List all Facility Ports on the site

In [ ]:
output_list = fablib.list_facility_ports(filter_function=lambda x: x['site_name'] == facility_port_site, output=cell_output)

#### Select the Facility Port on the site

Set the facility port name and select the VLAN to use

In [ ]:

facility_port_vlan='3919'

site='FIU'
worker1=f'{site.lower()}-w3.fabric-testbed.net'
worker2=f'{site.lower()}-w1.fabric-testbed.net'

cores=2
ram=4
disk=10

### Create Slice

Create new_slice and add node(s) to the FABRIC topology

In [ ]:
slice = fablib.new_slice(name=slice_name)

# Example: One VM on FABRIC-STAR with a NIC - sharedNIC (basic) or dedicated smartNIC (ConnectX_6)
node = slice.add_node(name=f"Node1", site=site, host=worker1, cores=cores, ram=ram, disk=disk, image="default_rocky_9")

#node_iface = node.add_component(model='NIC_ConnectX_6', name="nic1").get_interfaces()[0]
node_iface = node.add_component(model='NIC_Basic', name="nic1").get_interfaces()[0]

Add facility port to the network service configuration along with the node interface(s)

In [ ]:
facility_port = slice.add_facility_port(name=facility_port_name, site=facility_port_site, vlan=facility_port_vlan)
facility_port_interface =facility_port.get_interfaces()[0]

#net = slice.add_l2network(name=f'net_facility_port', interfaces=[node_iface,facility_port_interface])
net = slice.add_l2network(name=f'net_facility_port', interfaces=[])
net.add_interface(node_iface)
net.add_interface(facility_port_interface)

In [ ]:
print(f"facility_port.get_site(): {facility_port.get_site()}")
print(f"facility_port.get_fim_interface(): {facility_port.get_fim_interface()}")

Submit slice request to create

In [ ]:
slice.submit();

## Observe the Slice's Attributes


In [ ]:
slice = fablib.get_slice(slice_name)

In [ ]:
slice.show()
slice.list_nodes()
slice.list_networks()
slice.list_interfaces()


## Run the Experiment

Most experiments will require automated configuration and execution. You can use the fablib library to execute arbitrary commands on your node. 

The following code demonstrates how to use fablib to execute a "Hello, FABRIC" bash script. The library uses the bastion and VM keys defined at the top of this notebook to jump through the bastion host and execute the script.

In [ ]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

# Configure the interface(s) of the VM(s) with designated subnet
subnet = IPv4Network("192.168.1.0/24")
available_ips = list(subnet)[2:]

In [ ]:
node1 = slice.get_node(name=f"Node1")        
node1_iface = node1.get_interface(network_name=f'net_facility_port') 
node1_addr = available_ips.pop(99)
print(f"node1_addr: {node1_addr}")
node1_iface.ip_addr_add(addr=node1_addr, subnet=subnet)

stdout, stderr = node1.execute(f'ip addr show {node1_iface.get_os_interface()}')

stdout, stderr = node1.execute(f'sudo ip link set dev {node1_iface.get_physical_os_interface_name()} up')

stdout, stderr = node1.execute(f'sudo ip link set dev {node1_iface.get_os_interface()} up')

In [ ]:
# Ping the node(s) that are active on the Facility Port side (e.g 192.168.1.10)
node1 = slice.get_node(name=f"Node1")     
node1_iface = node1.get_interface(network_name=f'net_facility_port') 

stdout, stderr = node1.execute(f'ping -c 5 192.168.1.10')

## Delete the Slice

Please delete your slice when you are done with your experiment.

In [ ]:
slice.delete()